# Make working directory

In [ ]:
import os
os.mkdir(r'line stuff')
os.chdir(r'C:\Users\UWAdmin\line stuff')



## Import libraries

In [ ]:
from threading import *
import time 
from datetime import datetime

from datetime import timedelta


In [ ]:
#import rpy2.robjects as robjects #r-python interfacing

import rpy2.robjects as robjects
from pymeasure.instruments import list_resources

import ftd2xx  # Thorlabs MFF101
import ftd2xx.defines as constants
from ftd2xx import listDevices, getDeviceInfoDetail

from time import sleep




from newportxps import NewportXPS
from newportxps.XPS_C8_drivers import XPS, XPSException
# import pysftp

# from ophyd import EpicsSignal
from collections import OrderedDict
import yaqc
import os
import platform




import numpy as np
import csv


xpsd_remoteip = '192.168.254.254'
# xpsd_hostip ='192.168.0.254'
hostname = 'XPS-1b81'

xps = NewportXPS(xpsd_remoteip)
print(xps.status_report())


xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()



def pos_all():
    for sname, _ in xps.stages.items():
        print('{}: {}'.format(sname, xps.get_stage_position(sname)))
        
        
from flipper import *




In [ ]:
xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()

# Launch the spectrometer

In [ ]:
from spectra import *
capture_photo("start",2,1,0) 

# Import pressure related libraries

In [ ]:

from pressure import close_valve,open_valve,close_all,current_pressure,gopr
current_pressure()

# Import Laser power related libraries

In [ ]:
from pymeasure.instruments.lighthousephotonics import Sprout
laser_power = Sprout('COM4')
laser_power.adapter.connection.baud_rate = 19200
laser_power.adapter.connection.read_termination = '\r'
laser_power.adapter.connection.write_termination = '\r'
laser_power.power=0.01

# Position the motion controller

In [ ]:
#pos_all() tells about current position of the motion controller

pos_all()


xps.move_stage('XYZ.Pos1',8) #moves x axis
xps.move_stage('XYZ.Pos2',8) #moves y axis
xps.move_stage('XYZ.Pos3',8) #moves z axis

In [ ]:
mirror("on")

# Control the laser power and switch on/off the mirror

In [ ]:
mirror("off")

In [ ]:
laser_power.power=0.01

## Prepare initial Data 

In [ ]:

power=[]
time=[]
pressure=[]
gas=[]
print("Enter number of initial samples: ")
n=int(input())
for x in range(n):
    powr=np.random.randint(150,1000)
    tm=np.random.randint(1001,5000)
    pr=np.random.randint(100,1000)
    for i in range(9):
        power.append(powr)
        time.append(tm)
        pressure.append(pr)

    
from datetime import date

p=str(date.today())
# path=r'C:\Users\UWAdmin\line stuff'
import os
# os.chdir(path)
# os.mkdir('Campaign ' + p)
# os.chdir('Campaign ' + p)
row=['power','time','pressure','ratio']
with open('dataset.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
        writer.writerows(zip(power,time,pressure))

        
with open('dataset-pre.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
        writer.writerows(zip(power,time,pressure))

# os.chdir(path)

df2=pd.read_csv('dataset.csv')
df2=df2.drop_duplicates()
df2.head()
df2.to_csv('data.csv',index=False)

In [ ]:
# os.chdir(r'C:\Users\UWAdmin\line stuff')
os.getcwd()

# Adjust the z axis

In [ ]:

def adjust(start,stop,step):
    #st is the starting position of the z axis
    #step is the increment of the z axis
    line=1
#     import timeit
#     begin=8.7
#     start = timeit.default_timer()
    import time
    
     
    
    iteration=(stop-start)/step
    zz=[float(start)+(step*i) for i in range(int(iteration+1))]
    print(zz)
    intensity=[]
    for i in range(len(zz)):
        time.sleep(4)

        xps.move_stage('XYZ.Pos3',zz[i])
        time.sleep(1)
        p=capture_photo('adjust',i,line,1)
        intensity.append(p)
    mirror('off')
   # stop = timeit.default_timer()

    #print('Time: ', stop - start)  
#     print(intensity)

    import numpy as np
    ix=intensity.index(np.max(intensity))
    xps.move_stage('XYZ.Pos3',zz[ix])
#     print("adjusted z position is: ",zz[ix])
    return zz[ix]

In [ ]:
adjust(8.3,8.4,0.02)  # the z axis starts at 6, stops at 8 and increment is 0.2 

# Another option to adjust focusing

In [ ]:
pos_all()
mirror("off")

xps.move_stage('XYZ.Pos1',13) #moves x axis
xps.move_stage('XYZ.Pos2',13) #moves y axis
xps.move_stage('XYZ.Pos3',8.4) #moves z axis

In [ ]:
xps.move_stage('XYZ.Pos1',13) #moves x axis
xps.move_stage('XYZ.Pos2',13) #moves y axis
xps.move_stage('XYZ.Pos3',8)
st=8.3
mirror('on')
zz=[float(st)+(0.02*i) for i in range(7)]
for i in range(len(zz)):
    xps.move_stage('XYZ.Pos3',zz[i])
    print("z axis at: ",zz[i])
    time.sleep(8)
    
mirror("off")

## Define thread class

In [ ]:
class Aa(Thread):
#     def kill(self): 
#         self.killed = True
    def run(self):
#         for i in range(1):
        if d['ramptime'][0]<0.5:
            time.sleep(.5-d['ramptime'][0])
        print("motion controller started")
        xps.run_trajectory('foreward',)
#             xps.move_stage('XYZ.X',5)
           # xps.move_stage('XYZ.Y',5)
        time.sleep(total_time)
        laser_power.power=0.01
        print("finisherd and current position is:\n")
        pos_all()
#         while(True):
#             global stop_threads 
#             if stop_threads: 
#                 break
        
class Bb(Thread):
#     def kill(self): 
#         self.killed = True
    def run(self):
        for i in range(2):
           
            if i==0:
                if d['ramptime'][0]<0.5:
                    print("mirror on")
        #                 start_time = datetime.now()
                    start_time = time.monotonic()
                    mirror('on')
        #                 end_time = datetime.now()
                    end_time = time.monotonic()
        #                 print("time to put the mirror on: ",start_time-end_time)
                    print(timedelta(seconds=end_time - start_time))
                    time.sleep(d['ramptime'][1]) #time for linear line    

                else:
                    time.sleep(mirror_sleep)
                
                    print("mirror on")
    #                 start_time = datetime.now()
                    start_time = time.monotonic()
                    mirror('on')
    #                 end_time = datetime.now()
                    end_time = time.monotonic()
    #                 print("time to put the mirror on: ",start_time-end_time)
                    print(timedelta(seconds=end_time - start_time))
                    time.sleep(d['ramptime'][1]) #time for linear line
            else:
                mirror('off')
                print("mirror off")

In [ ]:
def take_median(i,idx):
    
    print(os.getcwd())

    df=pd.read_csv('dataset.csv')

    lth=len(df['ratio'])
    k=i
    median=[]

    valss=np.sort([df['ratio'][k+8],df['ratio'][k+7],df['ratio'][k+6],df['ratio'][k+5],df['ratio'][k+4],df['ratio'][k+3],
                   df['ratio'][k+2],df['ratio'][k+1],df['ratio'][k]])
    print(valss)

    lst=[s for s in valss if str(s)!='nan']
    print(valss[1])    
   # return valss[1]
    result=np.mean(lst)
    df2=pd.read_csv('data.csv')

    df2.loc[idx,"ratio"]=result
    df2.to_csv('data.csv',index=False)
    return result

# Adjust the pressure and check it

In [ ]:
for prsr in range(5): gopr(120)

## Define the line coordinate details

In [ ]:

df=pd.read_csv("dataset.csv")
df.head()


print("Enter number of lines, starting point for x axis, y axis and step size for y axis:")


lines,startx,starty,step_y= [x for x in input().split()]

print(lines,startx,starty,step_y)
move_y=[0 for i in range(int(lines))]
# print(move_y)

# move_y=[(float(move_y[i-1])+float(step_y)) if i!=0 else float(starty)  for i in range(int(lines))]


for i in range(int(lines)):
#     print(i)
    if i==0: move_y[i]=float(starty)
    if i>0 :
        move_y[i]=float(move_y[i-1])+float(step_y)
#         print(move_y[i-1]+float(step_y))


print("Lines will be made at following y axis:",move_y,"\n")



Enter number of lines, starting point for x axis, y axis and step size for y axis:
4 4.2 7 .5

In [ ]:
init_data=4
df2=pd.read_csv('data.csv')
print("Pateerning is to be started\n")
import time
time.sleep(5)
cc=0
ddd=pd.read_csv('dataset.csv')
steps=len(ddd)-10+1

for i in range(65,70):
    print("Line number: ",i)
    line=i
    adjust(8.3,8.42,0.02) 
    #move the axes to their start position
    
    xps.move_stage('XYZ.Pos2',move_y[i])
    pos_all()
    print("\n\n")
    df2=pd.read_csv('data.csv')
    time_of_file=df2['time'][i]
    #scantime=time_of_file/1000
    #define the trajectory
    
    xps.define_line_trajectories(start=float(startx),
                             stop=float(startx)+2,
                             step=0.01,
                             scantime=(time_of_file/1000),
                             axis='Pos1')
    
    print("\nCurrent time is:",time_of_file)
        
    pressure_of_file=df2['pressure'][i]
    for ppr in range(5):gopr(pressure_of_file)

    print("\n Pressure is now : ",pressure_of_file,"\n")
    
#     k+=3
    xps.download_trajectory('foreward.trj')
    import pandas as pd
    d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
    d = d.loc[:, (d != 0).any(axis=0)]
    d.columns = ['ramptime','rampdist','rampvel']

   
    
    
    total_time=np.sum(d['ramptime'])
    mirror_sleep=d['ramptime'][0]-.5
    
    
    #power will be set to assigned value and make sure mirror is off and power is given enough time to reach it's value
   
    power_of_file=df2["power"][i]
    ij=3
  

##################

    print("pre-pattern checking\n")
 #   check=[(float(startx)+.4),float(startx)+.8,(float(startx)+1.2)]

    a=0.19
    check=[ float(startx)+(a*i)+0.1 for i in range(10)]
    mm=0
    for kk in range(3):
        xps.move_stage('XYZ.Pos1',check[kk])
        print(pos_all())
        
        for iii in range(3):
            mirror("on")

            print("hey")
            GD=capture_photo("first",kk,line,iii)

            mirror("off")
            time.sleep(5)
    
 
############### 

#     print("pre-pattern checking done\n")
    print("power is now:",power_of_file)
    xps.move_stage('XYZ.Pos1',(float(startx)-d['rampdist'][0]))
    pos_all()
    time.sleep(15)
    print("\n\n")
    mirror("off")
    a=power_of_file
    laser_power.power=((a-2.7979)/0.4639)/1000
    time.sleep(15)
    
    ##Draw Lines
    time.sleep(5)
    
    
    
    
    import sys
    a=Aa()
    b=Bb()
    # stop_threads = False
    a.start()
    b.start()
    
    a.join()
    b.join()
    time.sleep(10)
    print("\n Job done\n")
    
    
    
    ##raman spectra analysis
    laser_power.power=0.01
    time.sleep(15)
    
    a=0.19
    check=[float(startx)+(a*i)+0.1 for i in range(10)]
    mm=0
#     check=[ float(startx)+(a*i)+0.1 for i in range(10)]
    ########################### post patterning
    print("line drawn, post pattern starts")
    
    for kk in range(3):
        xps.move_stage('XYZ.Pos1',check[kk])
        print(pos_all())
        
        for iii in range(3):
            mirror("on")
            #writing G/D of 3 spots in 3 lines inside dataset.csv
    #         capture_photo("on",kk,line)
            print("post patterning...")
            GD=capture_photo("on",kk,line,iii)

            mirror("off")
            time.sleep(5)      
     
    ##################################
    
    #so line 70--75 completed the raman spectra for 3 spots of a line.
   
    #now take the median out of them and write in dataset-2.csv
   
    #rto=take_median()
    if i<init_data-2:
        take_median(steps,line)
        steps+=9
        
    
    if i>init_data-1:
        mv=take_median(steps,line)
        steps+=9
        if mv is None:
            rsum((0))

        else:
            mv=float(mv)
            rsum((mv))
        write_more()
        repeats()
    

    if i>=init_data-1:
        print("AI started thinking")
        robjects.r('''
                sourin<-function(s){

                if(s==1){
                require(XML)

                  suppressWarnings({suppressMessages({
                    library(mlrMBO)
                    library(ggplot2)
                    library(doMC)
                  })})

                  ps = makeParamSet(
                    makeIntegerParam("power", lower = 151, upper =5000),
                    makeIntegerParam("time", lower = 1500, upper = 9000),
                    makeIntegerParam("pressure", lower = 100, upper = 980)

                  )

                  ctrl = makeMBOControl(y.name = "ratio")
                  ctrl = setMBOControlInfill(ctrl, opt = "focussearch",opt.focussearch.maxit = 10, opt.focussearch.points = 10000, crit =makeMBOInfillCritCB(cb.lambda=1))

                #dataset-2 should be replaced and later this line as well
                  data=read.csv("data.csv")

                data<-na.omit(data)
                  suppressMessages({opt.state = initSMBO(par.set = ps,design = data, control = ctrl, minimize = FALSE, noisy = TRUE)})

                  print("Proposed parameters:")
                  prop <- suppressWarnings({proposePoints(opt.state)})
                  print(prop$prop.points) 
                  print(prop$crit.components)
                  saveRDS(prop,'./prop.rds')
                  saveRDS(opt.state,'./opt.state.rds')
                  dummy_ratio=0
                  cat("Expected value (upper bound):\n")
                  cat(paste(prop$crit.components$mean, " (", \
                  prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))

                  x<-c(prop$prop.points,dummy_ratio,prop$crit.components$mean,\
                  prop$crit.components$mean + prop$crit.components$se)

                  x2<-c(prop$prop.points)
                  #data_plot=read.csv("plot_data.csv")


                  #dataset-2 should be replaced and later this line as well
                  write.table(x2, file = "data.csv", sep = ",",\
                  append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

                  #write.table(x, file = "plot_data.csv", sep = ",",\
                  #append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

                }
                else{
                  prop=readRDS('./prop.rds')
                  opt.state=readRDS('./opt.state.rds')
                  library(mlrMBO)
                  print(prop$crit.components)
                  print(prop$prop.points)
                  print("Proposed parameters:")
                  updateSMBO(opt.state, x = prop$prop.points, y = s)
                  prop <- suppressWarnings({proposePoints(opt.state)})
                  print(prop$prop.points) 
                  print(prop$crit.components)
                  saveRDS(prop,'./prop.rds')
                  saveRDS(opt.state,'./opt.state.rds')
                  dummy_ratio=0
                  cat("Expected value (upper bound):\n")
                  cat(paste(prop$crit.components$mean, " (", prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))
                  x<-c(prop$prop.points,dummy_ratio,prop$crit.components$mean,prop$crit.components$mean + prop$crit.components$se)
                  x2<-c(prop$prop.points)
                  #data_plot=read.csv("plot_data.csv")

                  #dataset-2 should be replaced and later this line as well
                  write.table(x2, file = "data.csv", sep = ",", append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)
                  #write.table(x, file = "plot_data.csv", sep = ",", append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

                }}

                ''')



    if i==init_data-1:
        rsum=robjects.r['sourin']
        rsum((1))
        write_more()
        repeats()

    



#  write_more() reads the latest proposal from data.csv and writes them in dataset.csv and dataset-pre.csv. The function repeats() writes two more times in both of the csvs.

In [ ]:
def write_more():
    d=pd.read_csv('data.csv')
    ln=d.shape[0]

    vpower=d['power'][ln-1]
    vtime=d['time'][ln-1]
    vpressure=d['pressure'][ln-1]

    d1=pd.read_csv('dataset.csv')
    ln=d1.shape[0]
    d1.loc[ln,"power"]=vpower
    d1.loc[ln,"time"]=vtime
    d1.loc[ln,"pressure"]=vpressure
    d1.to_csv('dataset.csv',index=False)
    d1.to_csv('dataset-pre.csv',index=False)

In [ ]:

def repeats():
    import csv
#     os.getcwd()
    d1=pd.read_csv('data.csv')
    df2=pd.read_csv('dataset.csv')
    ln=len(df2['power'])
    m=ln
    print(ln)
    counter=m
    for i in range(8):
        toAdd = [df2['power'][m-1],df2['time'][m-1],df2['pressure'][m-1]]
        filename="dataset.csv"
        with open(filename, "r") as infile:
            reader = list(csv.reader(infile))
            reader.insert(counter+1, toAdd)

        with open(filename, "w", newline='') as outfile:
            writer = csv.writer(outfile)
            for line in reader:
                writer.writerow(line)
                
    for i in range(8):
        toAdd = [df2['power'][m-1],df2['time'][m-1],df2['pressure'][m-1]]
        filename="dataset-pre.csv"
        with open(filename, "r") as infile:
            reader = list(csv.reader(infile))
            reader.insert(counter+1, toAdd)

        with open(filename, "w", newline='') as outfile:
            writer = csv.writer(outfile)
            for line in reader:
                writer.writerow(line)

# Switch off the laser after experiment is finished

In [ ]:
laser_power.write("OPMODE=OFF")